In [249]:
import pandas as pd
import numpy as np
import json
import geojson

In [250]:
#import climate change data
df = pd.read_csv('./raw/climate_change_orig.csv')

#delete not used columns
df.drop(['Domain Code', 'Area Code', 'Domain', 'Element Code', 'Element', 'Months Code', 'Months', 'Year Code', 'Unit', 'Flag', 'Flag Description'], axis=1, inplace=True)

#rename columns
df.columns = ['country_eng', 'year', 'value']

df.head()

,country_eng,year,value
0,Afghanistan,1961,-0.080
1,Afghanistan,1962,-0.120
2,Afghanistan,1963,0.882
3,Afghanistan,1964,-0.727
4,Afghanistan,1965,-0.200


In [251]:
#import country codes for mapping
df_country_code = pd.read_csv('./raw/country_codes.CSV', index_col=5)

#delete not used columns
df_country_code.drop(['LANG', 'LANG_NAME', 'COUNTRY_ALPHA3_CODE', 'COUNTRY_NUMERIC_CODE'], axis=1, inplace=True)

#create dict: (<countryname>: <countycode>)
country_dict = df_country_code.to_dict('dict')


In [252]:
def get_country_code(country_name, country_dict):
    """returns the country code of specific country
    
        Parameters:
        country_name: Name of the country in english (String)
        country_dict: Dictionary with country - country code mapping (Dictionary)

        Returns: country code if found, otherwise empty string (String)

       """    

    #handling countries with different names
    if (country_name == "Democratic People's Republic of Korea"):
        return "kp"
    if (country_name == "Republic of Korea"):
        return "kr"
    if (country_name == "French Southern and Antarctic Territories"):
        return "tf"
    if (country_name == "Ethiopia PDR"):
        return "et"
    if (country_name == "Midway Island"):
        return "mi"
    if (country_name == "Netherlands Antilles (former)"):
        return "an"
    if (country_name == "Pacific Islands Trust Territory"):
        return "pc"
    if (country_name == "Pitcairn Islands"):
        return "pn"
    if (country_name == "Republic of Moldova"):
        return "md"
    if (country_name == "Serbia and Montenegro"):
        return "cs"
    if (country_name == "Sudan (former)"):
        return "sd"
    if (country_name == "Svalbard and Jan Mayen Islands"):
        return "sj"
    if (country_name == "United Republic of Tanzania"):
        return "tz"
    if (country_name == "United States Virgin Islands"):
        return "vi"
    if (country_name == "Wake Island"):
        return "wk"
    if (country_name == "Wallis and Futuna Islandss"):
        return "wf"
    if (country_name == "Yugoslav SFR"):
        return "yu"
    if (country_name == "Belgium-Luxembourg"):
        return "be"
    if (country_name == "Yugoslav SFR"):
        return "yu"
    if (country_name == "Channel Islands"):
        return "ky"
    if (country_name == "China, Hong Kong SAR"):
        return "hk"
    if (country_name == "China, Macao SAR"):
        return "mo"
    if (country_name == "China, mainland"):
        return "cn"
    if (country_name == "China, Taiwan Province of"):
        return "tw"
    if (country_name == "Wallis and Futuna Islands"):
        return "wf"
    if (country_name == "Kosovo"):
        return "xk"
    if (country_name == "Namibia"):
        return "na"
    if (country_name == "Norway"):
        return "-99"
    if (country_name == "France"):
        return "fx"
    if (country_name == "Somaliland"):
        return "so"


    for key, value in country_dict.get('COUNTRY_ALPHA2_CODE').items():
        if key == country_name:
            return str(value).lower()


    return "NO_CC"

In [253]:
#mapping (country -> country code) & add to dataframe
df['country_code'] = df['country_eng'].apply(lambda x: get_country_code(x, country_dict))

In [254]:
def get_country_translation(country_code, country_dict):
    """returns the translated country name
    
        Parameters:
        country_code: Alpha2 Country Code (String)
        country_dict: Dictionary with country code - translated country name mapping (Dictionary)

        Returns: translated country name if found, otherwise empty String (String)

       """    

    for key, value in country_dict.items():
        if key in country_code:
            return str(value)
    return "NO_TRANSLATION"

In [255]:
#import french translated country name
df_de = pd.read_csv('./raw/country_translation_de.csv', usecols=['name', 'alpha2'])
df_fr = pd.read_csv('./raw/country_translation_fr.csv', usecols=['name', 'alpha2'])
df_it = pd.read_csv('./raw/country_translation_it.csv', usecols=['name', 'alpha2'])

#create dict: (<countycode>:<translated countryname>)
country_dict_de = df_de.set_index('alpha2')['name'].to_dict()
country_dict_fr = df_fr.set_index('alpha2')['name'].to_dict()
country_dict_it = df_it.set_index('alpha2')['name'].to_dict()

In [256]:
#mapping (country code -> translated country name) & add to dataframe
df['country_de'] = df['country_code'].apply(lambda x: get_country_translation(x, country_dict_de))
df['country_fr'] = df['country_code'].apply(lambda x: get_country_translation(x, country_dict_fr))
df['country_it'] = df['country_code'].apply(lambda x: get_country_translation(x, country_dict_it))

#no translation: Anguilla, ai, Antarctica, aq, Aruba, aw, China, Taiwan Province of, tw, Christmas Island, cx, Cocos (Keeling) Islands, cc, Faroe Islands, fo, French Guiana, gf, French Polynesia, pf, French Southern and Antarctic Territories, tf, Gibraltar, gi, Greenland, gl, Guadeloupe, gp, Holy See, va, Isle of Man, im, Martinique, mq, Mayotte, yt, Montserrat, ms, New Caledonia, nc, Niue, nu, Norfolk Island, nf, Palestine, ps, Pitcairn Islands, pn, Puerto Rico, pr, Réunion, re, Saint Helena, Ascension and Tristan da Cunha, sh, Saint Pierre and Miquelon, pm,South Georgia and the South Sandwich Islands, gs, Svalbard and Jan Mayen Islands, sj, Tokelau, tk, Turks and Caicos Islands, tc, Wallis and Futuna Islands, wf, Western Sahara, eh

df.head()

,country_eng,year,value,country_code,country_de,country_fr,country_it
0,Afghanistan,1961,-0.080,af,Afghanistan,Afghanistan,Afghanistan
1,Afghanistan,1962,-0.120,af,Afghanistan,Afghanistan,Afghanistan
2,Afghanistan,1963,0.882,af,Afghanistan,Afghanistan,Afghanistan
3,Afghanistan,1964,-0.727,af,Afghanistan,Afghanistan,Afghanistan
4,Afghanistan,1965,-0.200,af,Afghanistan,Afghanistan,Afghanistan


In [257]:
#save as a new csv file
df.to_csv("./climate_change_cleaned.csv")

In [258]:
#open geojson default db
with open('./raw/ne_110m_admin_0_countries.geojson', 'r') as f:
    geodata = geojson.load(f)

In [259]:
def get_temperature(country_code, country_name, df):
    """returns the temperature of specific country with country code
    
        Parameters:
        country_code: ISO alpha 2 Country Code (String)
        df: dataframe with country code and temperature values (Dataframe)

        Returns: temperature value if found, otherwise 999 (Str)

       """  
    code =  country_code 

    #Handle countries with not existing country code
    if (country_code == "-99" and country_name == "France"):
            code = "fx"

    for index, row in df.iterrows():
        if row['country_code'].lower() in code.lower() and row['year'] == 2019:
            return str(row['value'])
    return "NO_DATA"

In [260]:
#add temp value to geojson properties
for feat in geodata['features']:
    feat['properties']['TEMP'] = get_temperature(feat['properties']['ISO_A2'], feat['properties']['ADMIN'], df)

In [261]:
#Write result to a new file
with open('./geojson_temperature.geojson', 'w') as f:
    geojson.dump(geodata, f)

In [262]:
#No climate data: kosovo, Burundi, Falkland Islands, Rwanda, Yemen